In [24]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
import json
# from cma.music.DataQueryClient import DataQueryClient


class warring_alert():
    def __init__(self,rain_type):
#         self.rs = redis.Redis(host='127.0.0.1', port=6379,password="tzqxj58660")
#         self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
#         self.con = pymssql.connect("172.21.158.201","down","downx","ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"   
        self.rain_type = rain_type
        self.now = self.get_latest()
    def get_rain(self):
        rain_dir = {
            "10min":"warring_zdz",
            "rain01":"rain01",
            "rain03":"rain03",
            "rain12":"rain12",
            "rain24":"rain24"        
        }
        if self.rain_type =="10min":
            data = self.now
            rain = data[(data['PRE']>0)&(data['PRE']<5000)][['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','PRE']]
            rain_data = rain.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti'])['PRE'].sum().reset_index()
        else:
            data = pd.read_csv("/workspace/liyuan3970/Data/My_Git/web_met/static/data/downfile/rain01.csv")
            rain_data = data
        return rain_data
    def get_latest(self):
        data = pd.read_csv("/workspace/liyuan3970/Data/My_Git/web_met/static/data/downfile/server.csv")
        data = data.astype({'Lat': 'float', 'Lon': 'float','PRE': 'float','WIN_S_Inst_Max': 'float', 'WIN_D_INST_Max': 'float','TEM':'float','VIS_HOR_1MI':'float'})
        return data         
    def get_wind(self):
        data = self.now
        wind = data[(data['WIN_S_Inst_Max']>17)&(data['WIN_S_Inst_Max']<5000)][['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','WIN_S_Inst_Max','WIN_D_INST_Max']]
        wind_data = wind.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','WIN_D_INST_Max'])['WIN_S_Inst_Max'].max().reset_index().sort_values('WIN_S_Inst_Max', ascending=False).drop_duplicates(subset=['Station_Id_C'], keep='first')
        return wind_data
    def get_temp(self):
        data = self.now
        tmin = data[(data['TEM']<0)&(data['TEM']<5000)][['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','TEM']]
        tmax = data[(data['TEM']>35)&(data['TEM']<5000)][['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','TEM']]
        tmax_data = tmax.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti'])['TEM'].max().reset_index().to_json(orient = "records", force_ascii=False)
        tmin_data = tmin.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti'])['TEM'].min().reset_index().to_json(orient = "records", force_ascii=False)
        return tmax_data,tmin_data
    def get_view(self):
        data = self.now
        view = data[(data['VIS_HOR_1MI']<1000)&(data['VIS_HOR_1MI']<5000)][['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti','VIS_HOR_1MI']]
        view_data = view.groupby(['Cnty','Province','Town','Station_Name','City','Station_Id_C','Lat','Lon','Alti'])['VIS_HOR_1MI'].min().reset_index().to_json(orient = "records", force_ascii=False)
        return view_data
    def get_radar(self):
        '''获取雷达数据'''
        img = pickle.loads(self.rs.get("radar"))
        return img
    def warring_data(self):
        # 开始编写风雨数据模型
#         radar = self.get_radar()
        rain = self.get_rain()
        wind = self.get_wind()
        tmax_data,tmin_data = self.get_temp()
        view_data = self.get_view()
        rain_data = rain.to_json(orient = "records", force_ascii=False)
        wind_data = wind.to_json(orient = "records", force_ascii=False)
        return rain_data,wind_data,tmax_data,tmin_data,view_data


rain_type = "10min"
worker = warring_alert(rain_type)
worker.warring_data()


CPU times: user 127 ms, sys: 0 ns, total: 127 ms
Wall time: 133 ms


('[{"Cnty":"临安区","Province":"浙江省","Town":"龙岗镇","Station_Name":"大峡谷村","City":"杭州市","Station_Id_C":"KA023","Lat":30.2364,"Lon":119.1022,"Alti":244.0,"PRE":0.1},{"Cnty":"云和县","Province":"浙江省","Town":"元和街道","Station_Name":"霞晓桥村","City":"丽水市","Station_Id_C":"K7872","Lat":28.1342,"Lon":119.6019,"Alti":120.0,"PRE":0.1},{"Cnty":"兰溪市","Province":"浙江省","Town":"水亭畲族乡","Station_Name":"水亭","City":"金华市","Station_Id_C":"K6087","Lat":29.185,"Lon":119.2794,"Alti":52.0,"PRE":0.1},{"Cnty":"兰溪市","Province":"浙江省","Town":"诸葛镇","Station_Name":"双牌水库","City":"金华市","Station_Id_C":"K6218","Lat":29.23,"Lon":119.25,"Alti":77.0,"PRE":0.1},{"Cnty":"北仑区","Province":"浙江省","Town":"春晓街道","Station_Name":"春晓","City":"宁波市","Station_Id_C":"K2316","Lat":29.7847,"Lon":121.875,"Alti":18.5,"PRE":0.1},{"Cnty":"天台县","Province":"浙江省","Town":"泳溪乡","Station_Name":"外婆湾","City":"台州市","Station_Id_C":"K8778","Lat":29.1558,"Lon":121.205,"Alti":181.3,"PRE":0.1},{"Cnty":"天台县","Province":"浙江省","Town":"雷峰乡","Station_Name":"大雷山","City":"台州市",

,Unnamed: 0,Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,...,WIN_S_Avg_2mi,TEM,DPT,RHU,VAP,PRE,Snow_Depth,VIS_HOR_1MI,CLO_Cov,CLO_Height_LoM
0,0,繁昌区,安徽省,NaN,14.0,宁芜上行K117应用（交通）,芜湖市,I5711,31.0500,118.2167,...,NaN,NaN,NaN,NaN,NaN,999998.0,NaN,3417.0,NaN,NaN
1,1,婺源县,江西省,NaN,14.0,婺源江湾镇,上饶市,J2207,29.4150,118.0253,...,0.1,25.9,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2,湾沚区,安徽省,NaN,12.0,湾沚,芜湖市,58338,31.1197,118.5947,...,1.0,26.0,25.3,96.0,32.3,0.0,NaN,4663.0,NaN,NaN
3,3,庐山市,江西省,NaN,14.0,庐山仰天坪,九江市,J1039,29.5344,115.9550,...,0.0,21.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,4,潜山市,安徽省,NaN,14.0,潜山余井天明,安庆市,I6781,30.7789,116.6003,...,0.2,25.2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25374,25374,淳安县,浙江省,王阜乡,14.0,光明村,杭州市,K1304,29.8011,118.9078,...,999998.0,25.4,999998.0,999998.0,999998.0,0.0,999998.0,999998.0,999998.0,999998.0
25375,25375,椒江区,浙江省,大陈镇,11.0,大陈,台州市,58666,28.4514,121.9033,...,5.1,28.1,27.9,99.0,37.6,0.0,NaN,21485.0,NaN,NaN
25376,25376,椒江区,浙江省,大陈镇,11.0,大陈,台州市,58666,28.4514,121.9033,...,4.7,28.1,27.9,99.0,37.6,0.0,NaN,20861.0,NaN,NaN
25377,25377,仙居县,浙江省,白塔镇,14.0,茶溪,台州市,K8929,28.7750,120.5811,...,0.0,27.8,27.3,97.0,36.2,0.0,999998.0,20000.0,999998.0,999998.0
